# Odporúčanie

In [14]:
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# dependencies
import pandas as pd
import wandb

In [16]:
#local modules
import sys
sys.path.append("..")

from _helpers import constants
from _helpers.verify_submission.verify_subm import main as verify_subm
from _helpers.score_submission.score_subm import main as score_subm

from models.model_random import ModelRandom
from models.model_nochange import ModelNoChange
from models.model_popular import ModelPopular
from models.model_log_reg import ModelLogisticRegression
from models.model_cheapest import ModelCheapest
from models.model_xgboost import ModelXGBoost

Iitialize wandb - logging tool

In [17]:
wandb.login()

True

Load the correct data

In [18]:
train_file = constants.DROPPED_TRAIN
test_file = constants.DROPPED_TEST

if (constants.SUBSET is not None):
    train_file = constants.DROPPED_SUBSET(constants.SUBSET, 'train')
    test_file = constants.DROPPED_SUBSET(constants.SUBSET, 'test')

df_train = pd.read_parquet(train_file)
df_test = pd.read_parquet(test_file)

In [19]:
models = {
    'random': ModelRandom(),
    'nochange': ModelNoChange(),
    'popular': ModelPopular(),
    'log-reg': ModelLogisticRegression(),
    'cheapest': ModelCheapest(),
    'xgboost': ModelXGBoost(),
}

Choose the correct model and setup parameters of the model

In [20]:
# Tinker with the parameters
run = 1
notes = 'Preprocessing from file, memory optimized'
params = {
    'model': 'xgboost',
    'subset': constants.SUBSET,
    'features': [
        "impressed_item_position",
        # "impressed_item_rating",
        # "user_impressed_item_interaction_count",
        "price",
        "price_above_impression_mean",
        "is_last_interacted",
    ]
}

wandb_run = wandb.init(entity='mcfreddie777', project="dp-recsys", name=f'model_{params["model"]}_run_{run}',
                       notes=notes)
wandb_run.config.update(params)

model = models[params['model']]
model.update(params)
model.fit(df_train)

Predikujeme odporúčania

In [21]:
df_recommendations = model.predict(df_test)

Verifikujeme predikcie

In [22]:
verify_subm(df_subm=df_recommendations, df_test=df_test)

Checking for required columns in the submission file...
> check passed
Checking for duplicate sessions in the submission file...
> check passed
Checking that all the required sessions are present in submission...
> check passed
All checks passed


In [23]:
df_recommendations.to_csv(constants.OUTPUT_DIR / f'submission_popular_{params["model"]}_{run}.csv', index=False)

Vypočítame si MRR na dátach

In [24]:
df_gt = pd.read_csv(constants.GROUND_TRUTH)
df_test = pd.read_parquet(test_file)
mrr, map3 = score_subm(df_subm=df_recommendations, df_gt=df_gt)
wandb_run.log({"mrr": mrr, "map3": map3})

Mean reciprocal rank:      0.2238
Mean average precision @3: 0.0727


In [25]:
wandb_run.finish()


map3,▁
mrr,▁
map3,0.0727
mrr,0.2238
